In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
training=pd.read_hdf('training.h5')

In [3]:
arr=training["close"].as_matrix()

In [4]:
itera=arr[0:1000000]

In [5]:
maxprice = training['close'].max()
minprice = training['close'].min()
priceDsize = int(round(maxprice - minprice))

m = 10
k = 5
posDsize = 2*m+1
actionDsize = 2*k+1

actions = list(range(-k,k+1))

def price2index(x):
    return (int(round((x-minprice)*100)))
def pos2index(x):
    return x+10

In [6]:
# define Q table 
q = np.ndarray(shape=(1+priceDsize*100,posDsize,actionDsize),dtype=float)
q.fill(np.inf)

In [7]:
class PnL:
    def __init__(self):
        #tracks current amount of symbol
        self.quantity = 0
        #cumulative sum of positions (net position)
        self.running_quantity=0
        #total negative from buying positions
        self.cost = 0.0
        #current market value of positions
        self.market_value = 0.0
        #realized pnl
        self.r_pnl = 0.0
        #average fill price of all positions
        self.average_price = 0.0
        #realized plus unrealized pnl is total pnl (ie what we want)
        self.total_pnl=0.0
    #function that processes fill statements
    def fill(self, n_pos, exec_price):
        #position delta from new buys
        pos_change = n_pos - self.quantity
        #buy or sell
        direction = np.sign(pos_change)
        #previous positioning
        prev_direction = np.sign(self.quantity)
        #quantity we end up with
        qty_closing = min(abs(self.quantity), abs(pos_change)) * direction if prev_direction != direction else 0
        #quantity we started with
        qty_opening = pos_change if prev_direction == direction else pos_change - qty_closing
        
        #updated cost with new positions
        new_cost = self.cost + qty_opening * exec_price
        #if the old position wasnt zero, modify cost for new amount *avg price
        #same for realized pnl
        if self.quantity != 0:
            new_cost += qty_closing * self.cost / self.quantity
            self.r_pnl += qty_closing * (self.cost / self.quantity - exec_price)

        #update current positioning, total positioning, cost
        self.quantity = n_pos
        self.running_quantity=n_pos+self.running_quantity
        self.cost = new_cost

    #update for market price and publish total_pnl
    def update(self, price):
        if self.quantity != 0:
            self.average_price = self.cost / self.quantity
        else: 
            self.average_price = 0
        self.market_value = self.quantity * price
        upl=self.market_value - self.cost
        self.total_pnl=upl+self.r_pnl
        return self.total_pnl

In [8]:
kappa=1e-4
gamma=0.999
alpha=0.001
epsilon=0.1 #run without initially, then with
converge=1e-3
delta = np.inf
firstprice=arr[0]

In [14]:
counter = 0
lod = []
while True:
    cumpos = 0
    state = [price2index(firstprice),pos2index(cumpos)]
    last=arr[0]
    pastpnl=0
    btc=PnL()
    for i in range(len(arr[1:-2])):
        current_price=last
        action = np.argmax(q[tuple(state)])
        state.append(action)
        chpos = actions[action]
        cumpos += chpos
        if cumpos > 10:
            chpos = 0
            cumpos = 10
        elif cumpos < -10:
            chpos = 0
            cumpos = -10
        #if we are taking a new position, we need to assign rewards
        btc.fill(chpos,current_price)
        new_price=arr[i]
        pnl=btc.update(new_price)
        delta_v=pnl-pastpnl
        reward=delta_v-0.5*kappa*delta_v**2
        pastpnl=pnl
        # define next state and append to state list
        prevstate = state
        state = [price2index(new_price),pos2index(cumpos)]
        last=new_price
        if q[tuple(prevstate)] == np.inf: # prevstate is state,action
            prevq = 0
        else:
            prevq = q[tuple(prevstate)]
        if np.max(q[tuple(state)]) == np.inf:
            thisq = 0
        else:
            thisq = np.max(q[tuple(state)])
        wholeqprev = q
        q[tuple(prevstate)] = prevq + alpha*(reward+gamma*thisq-prevq)
    deltas = np.absolute(np.ravel(np.subtract(q,wholeqprev)))
    delta = np.max(deltas[~np.isnan(deltas)])
    lod.append(delta)
    counter += 1
    print('run count: '+str(counter))

KeyboardInterrupt: 

In [13]:
delta

inf